In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
wheels_path = "/kaggle/input/clip-interrogator-0-5-4"
clip_interrogator_whl_path = f"{wheels_path}/clip_interrogator-0.5.4-py3-none-any.whl"
!pip install --no-index --find-links $wheels_path  $clip_interrogator_whl_path #-q !pip install --no-index /kaggle/input/pytorchtransformers
import sys
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
sys.path.append('../input/pytorchtransformers')
!cp -r /kaggle/input/transformers-master/src/transformers/generation/ /opt/conda/lib/python3.7/site-packages/transformers/

Looking in links: /kaggle/input/clip-interrogator-0-5-4
Processing /kaggle/input/clip-interrogator-0-5-4/clip_interrogator-0.5.4-py3-none-any.whl
Processing /kaggle/input/clip-interrogator-0-5-4/blip_ci-0.0.2-py3-none-any.whl
Processing /kaggle/input/clip-interrogator-0-5-4/open_clip_torch-2.14.0-py3-none-any.whl
Processing /kaggle/input/clip-interrogator-0-5-4/safetensors-0.2.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/clip-interrogator-0-5-4/fairscale-0.4.4-py3-none-any.whl
Processing /kaggle/input/clip-interrogator-0-5-4/timm-0.4.12-py3-none-any.whl
Processing /kaggle/input/clip-interrogator-0-5-4/ftfy-6.1.1-py3-none-any.whl
  Attempting uninstall: timm
    Found existing installation: timm 0.6.13
    Uninstalling timm-0.6.13:
      Successfully uninstalled timm-0.6.13


In [3]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration

In [4]:
#option for CPU
processor = AutoProcessor.from_pretrained("/kaggle/input/image-caption-models/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("/kaggle/input/image-caption-models/blip2-opt-2.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch

#bring e
sample_submission = pd.read_csv('/kaggle/input/stable-diffusion-image-to-prompts/sample_submission.csv', index_col = 'imgId_eId')
images = os.listdir('/kaggle/input/stable-diffusion-image-to-prompts/images')
image_ids = [i.split('.')[0] for i in images]
EMBEDDING_LENGTH = 384
eIds = list(range(EMBEDDING_LENGTH))
imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(image_ids, EMBEDDING_LENGTH), # [인덱스 0부터 6 384번 반복]
        np.tile(range(EMBEDDING_LENGTH), len(image_ids)) # [0 ~ 383, 0 ~ 383, ......]
    )
]
def make_batches(l, batch_size=16):
    for i in range(0, len(l), batch_size):
        yield l[i:i + batch_size]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
BATCH_SIZE = 4
cap_list_blip2 = []
for ix, batch in enumerate(make_batches(images, BATCH_SIZE)):
    images_batch = []
    for i, image in enumerate(batch):
        images_batch.append(Image.open('/kaggle/input/stable-diffusion-image-to-prompts/images/'+image).convert('RGB'))
    pixel_values = processor(images = images_batch, return_tensors = 'pt').pixel_values.to(device)
    out = model.generate(pixel_values = pixel_values, max_length = 30, num_return_sequences = 1,
                       num_beams = 1, min_length = 10)
    prompts = processor.batch_decode(out, skip_special_tokens = True)
    for i in range(len(images_batch)):
        for j in range(1):
                caption = prompts[i * 1 + j]
                prompt = caption
                cap_list_blip2.append(prompt)
# for index, elem in enumerate(prompts):
# cap_list.append(clip_text[index]+', '+elem)
for i in cap_list_blip2:
    print(i)

del model

a man standing in front of a window with a sign that says it donuts don't

a cartoon dinosaur holding a piece of cheese in the woods

a drawing of a robot on paper with a black background

a large circular hole in the ground with a small hole in the middle

an astronaut standing in front of cherry blossoms

a circular object with a black base and a brown design

a painting of a man with a bird on his head



In [7]:
import inspect
import importlib


# replace tokenizer path to prevent downloading
blip_path = "/opt/conda/lib/python3.7/site-packages/blip/models/blip.py"

fin = open(blip_path, "rt")
data = fin.read()
data = data.replace(
    "BertTokenizer.from_pretrained('bert-base-uncased')", 
    "BertTokenizer.from_pretrained('/kaggle/input/clip-interrogator-models-x/bert-base-uncased')"
)

fin.close()

fin = open(blip_path, "wt")
fin.write(data)
fin.close()

from clip_interrogator import clip_interrogator
# fix clip_interrogator bug
clip_interrogator_path = inspect.getfile(clip_interrogator.Interrogator)

fin = open(clip_interrogator_path, "rt")
data = fin.read()
data = data.replace(
    'open_clip.get_tokenizer(clip_model_name)', 
    'open_clip.get_tokenizer(config.clip_model_name.split("/", 2)[0])'
)
fin.close()

fin = open(clip_interrogator_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(clip_interrogator)

from blip.models import blip
import os
import sys
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt 


import pandas as pd
import torch
import open_clip


comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')
class CFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    sentence_model_path = "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    blip_model_path = "/kaggle/input/clip-interrogator-models-x/model_large_caption.pth"
    ci_clip_model_name = "ViT-H-14/laion2b_s32b_b79k"
    clip_model_name = "ViT-H-14"
    clip_model_path = "/kaggle/input/clip-interrogator-models-x/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_pytorch_model.bin"
    cache_path = "/kaggle/input/vlomme-clip-iter"


model_config = clip_interrogator.Config(clip_model_name=CFG.ci_clip_model_name)
model_config.cache_path = CFG.cache_path
model_config.blip_num_beams = 1
configs_path = os.path.join(os.path.dirname(os.path.dirname(blip_path)), 'configs')
med_config = os.path.join(configs_path, 'med_config.json')
blip_model = blip.blip_decoder(
    pretrained=CFG.blip_model_path,
    image_size=model_config.blip_image_eval_size, 
    vit=model_config.blip_model_type, 
    med_config=med_config
)
blip_model.eval()
blip_model = blip_model.to(model_config.device)
model_config.blip_model = blip_model
clip_model = open_clip.create_model(CFG.clip_model_name, precision='fp16' if model_config.device == 'cuda' else 'fp32')
open_clip.load_checkpoint(clip_model, CFG.clip_model_path)
clip_model.to(model_config.device).eval()
model_config.clip_model = clip_model
clip_preprocess = open_clip.image_transform(
    clip_model.visual.image_size,
    is_train = False,
    mean = getattr(clip_model.visual, 'image_mean', None),
    std = getattr(clip_model.visual, 'image_std', None),
)
model_config.clip_preprocess = clip_preprocess
ci = clip_interrogator.Interrogator(model_config)
cos = torch.nn.CosineSimilarity(dim=1)

mediums_features_array = torch.stack([torch.from_numpy(t) for t in ci.mediums.embeds]).to(ci.device)
movements_features_array = torch.stack([torch.from_numpy(t) for t in ci.movements.embeds]).to(ci.device)
flavors_features_array = torch.stack([torch.from_numpy(t) for t in ci.flavors.embeds]).to(ci.device)

load checkpoint from /kaggle/input/clip-interrogator-models-x/model_large_caption.pth
Loaded CLIP model and data in 3.51 seconds.


In [8]:
BATCH_SIZE = 8
clip_text = []
cos = torch.nn.CosineSimilarity(dim=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for batch in make_batches(images, BATCH_SIZE):
  images_batch = []
  for i, image in enumerate(batch):
    images_batch.append(clip_preprocess(Image.open('/kaggle/input/stable-diffusion-image-to-prompts/images/'+image).convert('RGB')).unsqueeze(0))
  images_batch = torch.cat(images_batch, 0).to(device)

  with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = clip_model.encode_image(images_batch)
    image_features /= image_features.norm(dim = -1, keepdim = True)

  for i in range(len(image_features)):
    medium = [ci.mediums.labels[i] for i in cos(image_features[i], mediums_features_array).topk(1).indices][0]
    movement = [ci.movements.labels[i] for i in cos(image_features[i], movements_features_array).topk(1).indices][0]
    flaves = ', '.join([ci.flavors.labels[i] for i in cos(image_features[i], flavors_features_array).topk(3).indices])

    prompt = f'{medium}, {movement}, {flaves}'
    clip_text.append(prompt)
for i in clip_text:
  print(i)
del clip_model

a digital rendering, conceptual art, the mighty donut, donut, at the counter
an illustration of, sumatraism, mmmmm, buttercup eating pizza, pastry lizard
a screenprint, art brut, ((robot)), robot cat, robot design
concept art, conceptual art, crater, studying a hell open rift portal, abstract holescape
a portrait, space art, american astronaut in the forest, astronaut walking, american astronaut
a woodcut, op art, whorl, swirling around, wood art
a surrealist painting, magic realism, magic realism painting, surrealist oil painting, kent monkman


In [9]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("/kaggle/input/image-caption-models/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("/kaggle/input/image-caption-models/blip-image-captioning-base")

In [10]:
model.to(device)
BATCH_SIZE = 4
cap_list = []
for ix, batch in enumerate(make_batches(images, BATCH_SIZE)):
  images_batch = []
  for i, image in enumerate(batch):
    images_batch.append(Image.open('/kaggle/input/stable-diffusion-image-to-prompts/images/'+image).convert('RGB'))
  pixel_values = processor(images = images_batch, return_tensors = 'pt').pixel_values.to(device)
  out = model.generate(pixel_values = pixel_values, max_length = 20, num_return_sequences = 2,
                       num_beams = 2, min_length = 5)
  prompts = processor.batch_decode(out, skip_special_tokens = True)
  for i in range(len(images_batch)):
    for j in range(2):
      caption = prompts[i * 2 + j]
      prompt = caption + clip_text[BATCH_SIZE * ix + i]
      cap_list.append(prompt)
# for index, elem in enumerate(prompts):
# cap_list.append(clip_text[index]+', '+elem)
for i in cap_list:
  print(i)
print(len(cap_list))
del model

a man in a blue shirt is standing in front of a pizza ovena digital rendering, conceptual art, the mighty donut, donut, at the counter
a man in a blue shirta digital rendering, conceptual art, the mighty donut, donut, at the counter
a blue dinosaur with a piece of cheese on itan illustration of, sumatraism, mmmmm, buttercup eating pizza, pastry lizard
a blue dinosaur with a piece of bread on itan illustration of, sumatraism, mmmmm, buttercup eating pizza, pastry lizard
a drawing of a robot with a dog on ita screenprint, art brut, ((robot)), robot cat, robot design
a drawing of a robot with a dog on it's heada screenprint, art brut, ((robot)), robot cat, robot design
a close up of the surface of the surface of the planetconcept art, conceptual art, crater, studying a hell open rift portal, abstract holescape
a close up of the surface of the surface of the craterconcept art, conceptual art, crater, studying a hell open rift portal, abstract holescape
an astronaut in a space suit and a sp

In [11]:
from transformers import AutoProcessor, BlipForConditionalGeneration

processor = AutoProcessor.from_pretrained("/kaggle/input/image-caption-models/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("/kaggle/input/image-caption-models/blip-image-captioning-large").to(device)

In [12]:
model.to(device)
BATCH_SIZE = 4
cap_list_model_large = []
for ix, batch in enumerate(make_batches(images, BATCH_SIZE)):
  images_batch = []
  for i, image in enumerate(batch):
    images_batch.append(Image.open('/kaggle/input/stable-diffusion-image-to-prompts/images/'+image).convert('RGB'))
  pixel_values = processor(images = images_batch, return_tensors = 'pt').pixel_values.to(device)
  out = model.generate(pixel_values = pixel_values, max_length = 20, num_return_sequences = 2,
                       num_beams = 2, min_length = 5)
  prompts = processor.batch_decode(out, skip_special_tokens = True)
  for i in range(len(images_batch)):
    for j in range(2):
      caption = prompts[i * 2 + j]
      prompt = caption + clip_text[BATCH_SIZE * ix + i]
      cap_list_model_large.append(prompt)
# for index, elem in enumerate(prompts):
# cap_list.append(clip_text[index]+', '+elem)
for i in cap_list_model_large:
  print(i)
print(len(cap_list_model_large))
del model

there is a man standing in front of a counter with a cake on ita digital rendering, conceptual art, the mighty donut, donut, at the counter
there is a man standing in front of a counter with a cake and a cake on ita digital rendering, conceptual art, the mighty donut, donut, at the counter
cartoon dinosaur - like dinosaur - like dinosaur with a cheeseburger in the woodsan illustration of, sumatraism, mmmmm, buttercup eating pizza, pastry lizard
cartoon dinosaur - like dinosaur - like dinosaur with a cheeseburger in the background, there isan illustration of, sumatraism, mmmmm, buttercup eating pizza, pastry lizard
drawing of a robot toy robot with a robot on it's sidea screenprint, art brut, ((robot)), robot cat, robot design
drawing of a robot toy robot with a robot on it's side, with a pencil anda screenprint, art brut, ((robot)), robot cat, robot design
there is a picture of a circular shaped object in the middle of the pictureconcept art, conceptual art, crater, studying a hell ope

In [13]:
from sentence_transformers import SentenceTransformer, models
st_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')
submission_custom_1 = st_model.encode(cap_list).flatten()
submission_custom_2 = st_model.encode(cap_list_blip2).flatten()
submission_custom_3 = st_model.encode(cap_list_model_large).flatten()


submission_custom_1 = np.reshape(submission_custom_1, (-1, 2, 384)).mean(1).flatten()
submission_custom_2 = np.reshape(submission_custom_2, (-1, 1, 384)).mean(1).flatten()
submission_custom_3 = np.reshape(submission_custom_3, (-1, 2, 384)).mean(1).flatten()


submission = (np.array(submission_custom_1)
             +np.array(submission_custom_3)
             +np.array(submission_custom_2))
submission = pd.DataFrame({'imgId_eId': imgId_eId,
                           'val' : submission})
submission.head()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,imgId_eId,val
0,f27825b2c_0,0.052036
1,f27825b2c_1,0.235359
2,f27825b2c_2,-0.129432
3,f27825b2c_3,-0.094582
4,f27825b2c_4,-0.119766


In [14]:
images = os.listdir('/kaggle/input/stable-diffusion-image-to-prompts/images/')
imgIds = [i.split('.')[0] for i in images]
EMBEDDING_LENGTH = 384
eIds = list(range(EMBEDDING_LENGTH))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, EMBEDDING_LENGTH),
        np.tile(range(EMBEDDING_LENGTH), len(imgIds)))]

assert sorted(imgId_eId) == sorted(submission.imgId_eId)
ground_truth = pd.read_csv('/kaggle/input/stable-diffusion-image-to-prompts/prompts.csv')
ground_truth = pd.merge(pd.DataFrame(imgIds, columns = ['imgId']), ground_truth, 
                        on = 'imgId', how = 'left')
ground_truth_embeddings = st_model.encode(ground_truth.prompt).flatten()
gte = pd.DataFrame(
    index = imgId_eId,
    data = ground_truth_embeddings,
    columns = ['val']
).rename_axis('imgId_eId')

from scipy import spatial
vec1 = gte['val']
vec2 = submission['val']
cos_sim = 1 - spatial.distance.cosine(vec1, vec2)
print(cos_sim)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.5303403735160828


In [15]:
submission.to_csv("submission.csv", index=False)